In [25]:
from optimization.acs import ACS_VRP,ACS_TSP
from optimization.bso import BSO_VRP,BSO_TSP
from optimization.ga import GA_VRP
from optimization.sa import SA_VRP
from optimization.tabu_search import TS_VRP
from optimization.dka import DKA_VRP

from optimization.other.algorithm_woa import WOA_VRP

from optimization.aco import ACO_VRP,ACO_TSP
from optimization.haco import HACO_VRP,HACO_TSP

from optimization.koneksi import ConDB

import sql_connection
import json
import time
import random
import datetime
import copy
import pandas as pd
import numpy as np
import scipy.stats as stats

import matplotlib.pyplot as plt
from matplotlib.cm import get_cmap
from matplotlib.patches import FancyArrowPatch

import warnings
warnings.filterwarnings('ignore')

# Generate Data

In [2]:
def generate_data(n = 30, random_state = None):
    random.seed(random_state)
    query = """SELECT 
                    p.post_id,
                    p.post_type,
                    CASE
                        WHEN pj.pj_jam_buka = pj.pj_jam_tutup THEN "tutup"
                        ELSE "buka"
                    END AS is_operate
                FROM 
                    posts p
                LEFT JOIN
                    posts_jadwal pj
                    ON p.post_id = pj.pj_id_tempat AND pj.pj_hari = "minggu"
                """

    df_location = sql_connection.read_from_sql(query)
    
    tourid = df_location[(df_location['post_type']=="location")&
                         (df_location['is_operate'] != "tutup")]['post_id'].values.tolist()
    tourid = random.sample(tourid,n)
    idhotel = df_location[df_location['post_type']=="hotel"]['post_id'].values.tolist()
    idhotel = idhotel[random.randint(0,len(idhotel)-1)]
    
    db = ConDB()

    hotel = db.HotelbyID(idhotel)
    tur = db.WisatabyID(tourid)
    timematrix = db.TimeMatrixbyID(hotel._id,tourid)
    
    return hotel,tur,timematrix#,tourid,idhotel

# Analisis


In [3]:
def find_node(node_id,tour):
    for node in tour:
        if node._id == node_id:
            return node
    return False

def time_to_second(time):
    return (time.hour*3600)+(time.minute*60)+time.second

def diff_second_between_time(time_a,time_b):
    #input: time_a and time_b, datetime.time()
    #output: time_b - time_a, seconds (int)
    return time_to_second(time_b) - time_to_second(time_a)

def get_attribute_analysis(solution):
    days_needed = len(solution)
    poi_included = len(sum([sol["index"] for sol in solution],[]))
    avg_rating = sum(sum([sol["rating"] for sol in solution],[]))/poi_included
    total_tarif = sum(sum([sol["tarif"] for sol in solution],[]))
    total_duration = sum([diff_second_between_time(j[0],j[-1]) for j in [i['waktu'] for i in solution]])
    return days_needed,poi_included,avg_rating,total_tarif,total_duration

def get_analysis(model,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating):
    start = time.time()
    model.set_model(tour=tur,hotel=hotel,timematrix=timematrix,
                    travel_days=travel_days,
                    degree_waktu = dwaktu,degree_tarif = dtarif,degree_rating = drating)
    solution,fitness = model.construct_solution()[-2:]
    end = time.time()
    run_time = end-start
    
    daily_fitness = []
    for day in solution:
        daily_fitness.append(model.MAUT([day],use_penalty=False))
    days_needed,poi_included,avg_rating,total_tarif,total_duration = get_attribute_analysis(solution)
    results = {"solution":[sol["index"] for sol in solution],
               "fitness":fitness,
               "daily_fitness":daily_fitness,
               "days_needed":days_needed,
               "poi_included":poi_included,
               "avg_rating":avg_rating,
               "total_tarif":total_tarif,
               "total_duration":total_duration,
               "run_time":run_time}
    
    return results

In [4]:
def get_detail_node_on_solution(solution):
    db = ConDB()
    node_solution = []
    for i in solution:
        nodes = db.WisatabyID(i)
        node_solution.append(nodes)
    return node_solution

def visualize_solution(solution_node,hotel):
    # Sample VRP data
    vrp_data = [[hotel]+sol+[hotel] for sol in solution_node]
    
    # Generate random node coordinates
    node_coords = {node._id: (float(node.long), float(node.lat)) for node in set(node for route in vrp_data for node in route)}  

    # Number of routes (you can calculate this)
    num_routes = len(vrp_data)

    # Create the plot
    fig, ax = plt.subplots(figsize=(12,9))

    # Define the colors for routes using a colormap
    cmap = plt.get_cmap('tab20', num_routes)

    for i, route in enumerate(vrp_data):
        x = [node_coords[node._id][0] for node in route]
        y = [node_coords[node._id][1] for node in route]
        color = cmap(i)  # Get the color for the route
        # Plot the nodes (scatter plot)
        ax.scatter(x, y, c=color, label=f'Route {i + 1}', s=100)

        for j in range(len(x) - 1):
            # Make the arrow heads larger by increasing mutation_scale
            arrow = FancyArrowPatch(
                (x[j], y[j]), (x[j + 1], y[j + 1]),
                arrowstyle='->', mutation_scale=15, color=color
            )
            ax.add_patch(arrow)

    # Set node 159 (start and end point) to black
    start_x, start_y = node_coords[hotel._id]
    ax.scatter(start_x, start_y, c='k', label='Hotel', s=100)

    # Add labels for nodes
    for node, (x, y) in node_coords.items():
        ax.annotate(str(node), (x, y), textcoords="offset points", xytext=(0, 10), ha='center')

    # Add legend
    ax.legend()

    plt.show()



## General Performance (50 random nodes)

In [11]:
def result_to_xlsx_rd_nodes(result,random_ls,filename):
    df = pd.DataFrame()
    for method in result:
        res_method = result[method]
        df_method = {
            "random_seed" : random_ls,
            "fitness" : [],
            "fitness_1" : [],
            "fitness_2" : [],
            "fitness_3" : [],
            "poi_included" : [],
            "rating" : [],
            "tarif" : [],
            "total_duration" : [],
            "run_time" : [],
        }
        for res in res_method:
            df_method["fitness"].append(res['fitness'])
            df_method["fitness_1"].append(res['daily_fitness'][0])
            df_method["fitness_2"].append(res['daily_fitness'][1])
            df_method["fitness_3"].append(res['daily_fitness'][2])
            df_method["poi_included"].append(res['poi_included'])
            df_method["rating"].append(res['avg_rating'])
            df_method["tarif"].append(res['total_tarif'])
            df_method["total_duration"].append(res['total_duration'])
            df_method["run_time"].append(res['run_time'])
        df_method = pd.DataFrame(df_method)
        df_method['method'] = method
        df = pd.concat([df,df_method])
    df.to_excel(filename,index=False)

### All DOI 1

In [6]:
hotel,tur,timematrix,tourid,idhotel = generate_data(n=30,random_state=40)
travel_days = 3
dwaktu,dtarif,drating = 1,1,1
n = 1

In [7]:
woa_model = WOA_VRP(tourid, idhotel, dwaktu, dtarif, drating, travel_days, n)
solution, fitness = woa_model.construct_solution()

Iteration = 0 | best fitness = 0.72074
Iteration = 1 | best fitness = 0.72074
Iteration = 2 | best fitness = 0.72074
Duration,  0:00:33.859217 seconds
Fitness value : 0.7069449696062602
Duration: 31692
Cost: 250000
Rating: 4.585714285714286
results
[{'index': [62, 13, 15, 99, 35, 4, 29], 'waktu': ['08:00:00', '09:07:33', '10:21:45', '10:57:40', '12:36:46', '13:39:35', '16:14:10', '17:44:10', '18:41:07'], 'rating': [4.6, 4.8, 4.5, 4.3, 4.6, 4.7, 4.4], 'tarif': [0, 0, 0, 30000, 10000, 50000, 10000]}, {'index': [85, 20, 24, 28, 89, 17, 57, 43], 'waktu': ['08:00:00', '08:12:43', '10:20:15', '11:08:28', '13:56:42', '15:29:08', '16:35:31', '18:38:23', '19:38:23', '20:32:13'], 'rating': [4.5, 4.7, 4.4, 4.4, 4.6, 4.5, 4.4, 4.8], 'tarif': [0, 10000, 0, 115000, 0, 0, 15000, 0]}, {'index': [92, 44, 7, 8, 96, 60], 'waktu': ['08:00:00', '08:25:27', '09:43:36', '10:43:59', '12:49:31', '14:56:42', '15:56:42', '16:09:52'], 'rating': [4.8, 4.7, 4.7, 4.7, 4.5, 4.7], 'tarif': [0, 3000, 0, 7000, 0, 0]}]


In [8]:
random_ls = list(range(5,(5*50)+1,5))
random_node_analysis ={
    "acs_vrp": [],
    "aco_vrp": [],
    "bso_vrp": [],
    "ga_vrp": [],
    "ts_vrp": [],
    "sa_vrp": [],
    "dka_vrp": [],
}
for i in range(len(random_ls)):
    # setting
    hotel,tur,timematrix = generate_data(n=30,random_state=random_ls[i]) #random state 50 blacklist
    travel_days = 3
    dwaktu,dtarif,drating = 1,1,1
    
    print("random state = ",random_ls[i])
    
    acs_vrp = ACS_VRP(random_state=100)
    results_acs_vrp = get_analysis(acs_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    random_node_analysis["acs_vrp"].append(results_acs_vrp)
    
    aco_vrp = ACO_VRP(random_state=100)
    results_aco_vrp = get_analysis(aco_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    random_node_analysis["aco_vrp"].append(results_aco_vrp)
    
    bso_vrp = BSO_VRP(random_state=100)
    results_bso_vrp = get_analysis(bso_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    random_node_analysis["bso_vrp"].append(results_bso_vrp)
    
    ga_vrp = GA_VRP(random_state=100)
    results_ga_vrp = get_analysis(ga_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    random_node_analysis["ga_vrp"].append(results_ga_vrp)

    ts_vrp = TS_VRP(random_state=100)
    results_ts_vrp = get_analysis(ts_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    random_node_analysis["ts_vrp"].append(results_ts_vrp)

    sa_vrp = SA_VRP(random_state=100)
    results_sa_vrp = get_analysis(sa_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    random_node_analysis["sa_vrp"].append(results_sa_vrp)
    
    dka_vrp = DKA_VRP(random_state=100)
    results_dka_vrp = get_analysis(dka_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    random_node_analysis["dka_vrp"].append(results_dka_vrp)

random state =  5
random state =  10
random state =  15
random state =  20
random state =  25
random state =  30
random state =  35
random state =  40
random state =  45
random state =  50
random state =  55
random state =  60
random state =  65
random state =  70
random state =  75
random state =  80
random state =  85
random state =  90
random state =  95
random state =  100
random state =  105
random state =  110
random state =  115
random state =  120
random state =  125
random state =  130
random state =  135
random state =  140
random state =  145
random state =  150
random state =  155
random state =  160
random state =  165
random state =  170
random state =  175
random state =  180
random state =  185
random state =  190
random state =  195
random state =  200
random state =  205
random state =  210
random state =  215
random state =  220
random state =  225
random state =  230
random state =  235
random state =  240
random state =  245
random state =  250


In [9]:
random_node_analysis

{'acs_vrp': [{'solution': [[34, 66, 2, 50, 7, 52, 4, 9, 15, 10],
    [55, 65, 93, 22, 35],
    [90, 14, 26, 51, 74, 88]],
   'fitness': 0.6814583957272639,
   'daily_fitness': [0.7093750000000002,
    0.6255612334032145,
    0.6508756988120196],
   'days_needed': 3,
   'poi_included': 21,
   'avg_rating': 4.614285714285715,
   'total_tarif': 37000,
   'total_duration': 125100,
   'run_time': 4.7696614265441895},
  {'solution': [[39, 74, 89, 10, 22, 36, 21, 32],
    [34, 51, 66, 18, 2, 57],
    [12, 6, 44, 71, 49, 46]],
   'fitness': 0.6441722659237409,
   'daily_fitness': [0.6318430432645035,
    0.6022748688954441,
    0.6027654867256638],
   'days_needed': 3,
   'poi_included': 20,
   'avg_rating': 4.58,
   'total_tarif': 24500,
   'total_duration': 124620,
   'run_time': 3.170985221862793},
  {'solution': [[46, 2, 33, 74, 53, 32, 22],
    [81, 68, 8, 20, 85, 57, 36, 3, 15, 50],
    [39, 71, 93, 43]],
   'fitness': 0.6119772106974622,
   'daily_fitness': [0.6038376050080051,
    0.57

In [12]:
result_to_xlsx_rd_nodes(random_node_analysis,random_ls,"dka_random_nodes_alldoi_1.xlsx")

### Travel Duration 1, other 0

In [13]:
random_ls = list(range(5,(5*50)+1,5))
random_node_analysis ={
    "acs_vrp": [],
    "aco_vrp": [],
    "bso_vrp": [],
    "ga_vrp": [],
    "ts_vrp": [],
    "sa_vrp": [],
    "dka_vrp": [],
}
for i in range(len(random_ls)):
    # setting
    hotel,tur,timematrix = generate_data(n=30,random_state=random_ls[i]) #random state 50 blacklist
    travel_days = 3
    dwaktu,dtarif,drating = 1,0,0
    
    print("random state = ",random_ls[i])
    
    acs_vrp = ACS_VRP(random_state=100)
    results_acs_vrp = get_analysis(acs_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    random_node_analysis["acs_vrp"].append(results_acs_vrp)
    
    aco_vrp = ACO_VRP(random_state=100)
    results_aco_vrp = get_analysis(aco_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    random_node_analysis["aco_vrp"].append(results_aco_vrp)
    
    bso_vrp = BSO_VRP(random_state=100)
    results_bso_vrp = get_analysis(bso_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    random_node_analysis["bso_vrp"].append(results_bso_vrp)
    
    ga_vrp = GA_VRP(random_state=100)
    results_ga_vrp = get_analysis(ga_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    random_node_analysis["ga_vrp"].append(results_ga_vrp)

    ts_vrp = TS_VRP(random_state=100)
    results_ts_vrp = get_analysis(ts_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    random_node_analysis["ts_vrp"].append(results_ts_vrp)

    sa_vrp = SA_VRP(random_state=100)
    results_sa_vrp = get_analysis(sa_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    random_node_analysis["sa_vrp"].append(results_sa_vrp)
    
    dka_vrp = DKA_VRP(random_state=100)
    results_dka_vrp = get_analysis(dka_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    random_node_analysis["dka_vrp"].append(results_dka_vrp)

random state =  5
random state =  10
random state =  15
random state =  20
random state =  25
random state =  30
random state =  35
random state =  40
random state =  45
random state =  50
random state =  55
random state =  60
random state =  65
random state =  70
random state =  75
random state =  80
random state =  85
random state =  90
random state =  95
random state =  100
random state =  105
random state =  110
random state =  115
random state =  120
random state =  125
random state =  130
random state =  135
random state =  140
random state =  145
random state =  150
random state =  155
random state =  160
random state =  165
random state =  170
random state =  175
random state =  180
random state =  185
random state =  190
random state =  195
random state =  200
random state =  205
random state =  210
random state =  215
random state =  220
random state =  225
random state =  230
random state =  235
random state =  240
random state =  245
random state =  250


In [14]:
random_node_analysis

{'acs_vrp': [{'solution': [[55, 2, 7, 15, 52, 88, 35, 66, 83, 9],
    [51, 34, 30, 26, 74],
    [90, 99, 50, 14, 48, 22, 10, 4]],
   'fitness': 0.6480324074074074,
   'daily_fitness': [0.5108796296296296,
    0.42407407407407405,
    0.47777777777777775],
   'days_needed': 3,
   'poi_included': 23,
   'avg_rating': 4.556521739130434,
   'total_tarif': 104500,
   'total_duration': 121980,
   'run_time': 3.0116777420043945},
  {'solution': [[97, 67, 18, 32, 10, 89, 36, 21, 49],
    [6, 12, 66, 57, 74, 5, 46, 2],
    [80, 22, 99, 44, 48, 71]],
   'fitness': 0.6423611111111112,
   'daily_fitness': [0.48796296296296293,
    0.4726851851851852,
    0.44074074074074077],
   'days_needed': 3,
   'poi_included': 23,
   'avg_rating': 4.534782608695653,
   'total_tarif': 196000,
   'total_duration': 124920,
   'run_time': 4.785303354263306},
  {'solution': [[42, 33, 50, 36, 15, 3, 85, 53, 2],
    [46, 57, 74, 99, 20, 93, 43],
    [47, 8, 81, 48, 32, 5, 22]],
   'fitness': 0.6312114197530865,
   '

In [15]:
result_to_xlsx_rd_nodes(random_node_analysis,random_ls,"dka_random_nodes_dwaktu_1.xlsx")

### Tarif 1, other 0

In [16]:
random_ls = list(range(5,(5*50)+1,5))
random_node_analysis ={
    "acs_vrp": [],
    "aco_vrp": [],
    "bso_vrp": [],
    "ga_vrp": [],
    "ts_vrp": [],
    "sa_vrp": [],
    "dka_vrp": [],
}
for i in range(len(random_ls)):
    # setting
    hotel,tur,timematrix = generate_data(n=30,random_state=random_ls[i]) #random state 50 blacklist
    travel_days = 3
    dwaktu,dtarif,drating = 0,1,0
    
    print("random state = ",random_ls[i])
    
    acs_vrp = ACS_VRP(random_state=100)
    results_acs_vrp = get_analysis(acs_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    random_node_analysis["acs_vrp"].append(results_acs_vrp)
    
    aco_vrp = ACO_VRP(random_state=100)
    results_aco_vrp = get_analysis(aco_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    random_node_analysis["aco_vrp"].append(results_aco_vrp)
    
    bso_vrp = BSO_VRP(random_state=100)
    results_bso_vrp = get_analysis(bso_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    random_node_analysis["bso_vrp"].append(results_bso_vrp)
    
    ga_vrp = GA_VRP(random_state=100)
    results_ga_vrp = get_analysis(ga_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    random_node_analysis["ga_vrp"].append(results_ga_vrp)

    ts_vrp = TS_VRP(random_state=100)
    results_ts_vrp = get_analysis(ts_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    random_node_analysis["ts_vrp"].append(results_ts_vrp)

    sa_vrp = SA_VRP(random_state=100)
    results_sa_vrp = get_analysis(sa_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    random_node_analysis["sa_vrp"].append(results_sa_vrp)
    
    dka_vrp = DKA_VRP(random_state=100)
    results_dka_vrp = get_analysis(dka_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    random_node_analysis["dka_vrp"].append(results_dka_vrp)

random state =  5
random state =  10
random state =  15
random state =  20
random state =  25
random state =  30
random state =  35
random state =  40
random state =  45
random state =  50
random state =  55
random state =  60
random state =  65
random state =  70
random state =  75
random state =  80
random state =  85
random state =  90
random state =  95
random state =  100
random state =  105
random state =  110
random state =  115
random state =  120
random state =  125
random state =  130
random state =  135
random state =  140
random state =  145
random state =  150
random state =  155
random state =  160
random state =  165
random state =  170
random state =  175
random state =  180
random state =  185
random state =  190
random state =  195
random state =  200
random state =  205
random state =  210
random state =  215
random state =  220
random state =  225
random state =  230
random state =  235
random state =  240
random state =  245
random state =  250


In [17]:
random_node_analysis

{'acs_vrp': [{'solution': [[34, 93, 4, 35, 10, 50, 9],
    [2, 14, 38, 55, 88, 7, 26],
    [51, 66, 99, 74, 22, 15, 52]],
   'fitness': 0.8094339622641509,
   'daily_fitness': [0.5930817610062893,
    0.5921383647798742,
    0.5836477987421383],
   'days_needed': 3,
   'poi_included': 21,
   'avg_rating': 4.604761904761904,
   'total_tarif': 43000,
   'total_duration': 128340,
   'run_time': 1.987593173980713},
  {'solution': [[38, 34, 39, 22, 57, 10],
    [44, 32, 46, 18, 71, 49, 66],
    [6, 89, 36, 21, 74, 51, 2]],
   'fitness': 0.8072271386430678,
   'daily_fitness': [0.5752212389380531,
    0.6078171091445428,
    0.5980825958702065],
   'days_needed': 3,
   'poi_included': 20,
   'avg_rating': 4.5649999999999995,
   'total_tarif': 29500,
   'total_duration': 127380,
   'run_time': 4.700382471084595},
  {'solution': [[57, 46, 22, 50, 71, 85, 3, 36, 15],
    [38, 20, 8, 93, 43],
    [32, 74, 2, 68, 81, 53, 33]],
   'fitness': 0.791200215920603,
   'daily_fitness': [0.62218358831710

In [18]:
result_to_xlsx_rd_nodes(random_node_analysis,random_ls,"dka_random_nodes_dtarif_1.xlsx")

### Rating 1, other 0

In [19]:
random_ls = list(range(5,(5*50)+1,5))
random_node_analysis ={
    "acs_vrp": [],
    "aco_vrp": [],
    "bso_vrp": [],
    "ga_vrp": [],
    "ts_vrp": [],
    "sa_vrp": [],
    "dka_vrp": [],
}
for i in range(len(random_ls)):
    # setting
    hotel,tur,timematrix = generate_data(n=30,random_state=random_ls[i]) #random state 50 blacklist
    travel_days = 3
    dwaktu,dtarif,drating = 0,0,1
    
    print("random state = ",random_ls[i])
    
    acs_vrp = ACS_VRP(random_state=100)
    results_acs_vrp = get_analysis(acs_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    random_node_analysis["acs_vrp"].append(results_acs_vrp)
    
    aco_vrp = ACO_VRP(random_state=100)
    results_aco_vrp = get_analysis(aco_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    random_node_analysis["aco_vrp"].append(results_aco_vrp)
    
    bso_vrp = BSO_VRP(random_state=100)
    results_bso_vrp = get_analysis(bso_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    random_node_analysis["bso_vrp"].append(results_bso_vrp)
    
    ga_vrp = GA_VRP(random_state=100)
    results_ga_vrp = get_analysis(ga_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    random_node_analysis["ga_vrp"].append(results_ga_vrp)

    ts_vrp = TS_VRP(random_state=100)
    results_ts_vrp = get_analysis(ts_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    random_node_analysis["ts_vrp"].append(results_ts_vrp)

    sa_vrp = SA_VRP(random_state=100)
    results_sa_vrp = get_analysis(sa_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    random_node_analysis["sa_vrp"].append(results_sa_vrp)
    
    dka_vrp = DKA_VRP(random_state=100)
    results_dka_vrp = get_analysis(dka_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    random_node_analysis["dka_vrp"].append(results_dka_vrp)

random state =  5
random state =  10
random state =  15
random state =  20
random state =  25
random state =  30
random state =  35
random state =  40
random state =  45
random state =  50
random state =  55
random state =  60
random state =  65
random state =  70
random state =  75
random state =  80
random state =  85
random state =  90
random state =  95
random state =  100
random state =  105
random state =  110
random state =  115
random state =  120
random state =  125
random state =  130
random state =  135
random state =  140
random state =  145
random state =  150
random state =  155
random state =  160
random state =  165
random state =  170
random state =  175
random state =  180
random state =  185
random state =  190
random state =  195
random state =  200
random state =  205
random state =  210
random state =  215
random state =  220
random state =  225
random state =  230
random state =  235
random state =  240
random state =  245
random state =  250


In [20]:
random_node_analysis

{'acs_vrp': [{'solution': [[74, 88, 2, 7, 50, 9, 15, 35, 93],
    [52, 22, 34, 4, 64, 10],
    [90, 55, 99, 72, 14, 48, 66, 83]],
   'fitness': 0.8263385668276974,
   'daily_fitness': [0.5944444444444447,
    0.498148148148148,
    0.4458333333333331],
   'days_needed': 3,
   'poi_included': 23,
   'avg_rating': 4.595652173913043,
   'total_tarif': 179500,
   'total_duration': 128040,
   'run_time': 2.198029041290283},
  {'solution': [[46, 89, 12, 44, 18, 5, 57, 74],
    [39, 64, 48, 22, 2, 32, 36],
    [6, 67, 99, 34, 66, 10, 21, 49]],
   'fitness': 0.7749530327428877,
   'daily_fitness': [0.4770833333333333,
    0.4023809523809523,
    0.3583333333333334],
   'days_needed': 3,
   'poi_included': 23,
   'avg_rating': 4.569565217391304,
   'total_tarif': 142000,
   'total_duration': 129420,
   'run_time': 5.17795991897583},
  {'solution': [[46, 50, 74, 53, 2, 3, 22, 33],
    [64, 39, 81, 93, 43, 15, 36],
    [8, 20, 68, 48, 32, 5, 57]],
   'fitness': 0.7382270121853458,
   'daily_fitne

In [21]:
result_to_xlsx_rd_nodes(random_node_analysis,random_ls,"dka_random_nodes_drating_1.xlsx")

## Random DOI Combinations

In [40]:
def result_to_xlsx_rd_doi(result,random_doi,random_ls,filename):
    df_doi = pd.DataFrame()
    for doi_res in range(len(result)):
        df = pd.DataFrame()
        for method in result[doi_res]:
            res_method = result[doi_res][method]
            df_method = {
                "random_seed" : random_ls,
                "fitness" : [],
                "fitness_1" : [],
                "fitness_2" : [],
                "fitness_3" : [],
                "poi_included" : [],
                "rating" : [],
                "tarif" : [],
                "total_duration" : [],
                "run_time" : [],
            }
            for res in res_method:
                df_method["fitness"].append(res['fitness'])
                df_method["fitness_1"].append(res['daily_fitness'][0])
                df_method["fitness_2"].append(res['daily_fitness'][1])
                df_method["fitness_3"].append(res['daily_fitness'][2])
                df_method["poi_included"].append(res['poi_included'])
                df_method["rating"].append(res['avg_rating'])
                df_method["tarif"].append(res['total_tarif'])
                df_method["total_duration"].append(res['total_duration'])
                df_method["run_time"].append(res['run_time'])
            df_method = pd.DataFrame(df_method)
            df_method['method'] = method
            df = pd.concat([df,df_method])
        
        doi = random_doi[doi_res]
        df['dwaktu'] = doi[0]
        df['dtarif'] = doi[1]
        df['drating'] = doi[2]
        df = df[['dwaktu','dtarif','drating','method','random_seed','fitness','fitness_1','fitness_2','fitness_3',
                'poi_included','rating','tarif','total_duration','run_time']]
        df_doi = pd.concat([df_doi,df])
#     return df_doi
    df_doi.to_excel(filename,index=False)

In [41]:
random.seed(100)
doi_combi = [(np.round(random.uniform(0,1),2),np.round(random.uniform(0,1),2),np.round(random.uniform(0,1),2)) for i in range(10)]

In [42]:
doi_combi

[(0.15, 0.45, 0.77),
 (0.71, 0.73, 0.43),
 (0.8, 0.53, 0.08),
 (0.46, 0.05, 0.93),
 (0.95, 0.34, 0.31),
 (0.77, 0.2, 0.18),
 (0.19, 0.35, 0.63),
 (0.96, 0.21, 0.96),
 (0.56, 0.9, 0.82),
 (0.16, 0.65, 0.12)]

In [43]:
doi_combi_analysis = []
for doi in doi_combi:
    print("doi combination = ",doi)
    random_ls = list(range(5,(5*50)+1,5))
    random_node_analysis ={
        "acs_vrp": [],
        "aco_vrp": [],
        "bso_vrp": [],
        "ga_vrp": [],
        "ts_vrp": [],
        "sa_vrp": [],
        "dka_vrp": [],
    }
    for i in range(len(random_ls)):
        # setting
        hotel,tur,timematrix = generate_data(n=30,random_state=random_ls[i]) #random state 50 blacklist
        travel_days = 3
        dwaktu,dtarif,drating = 0,0,1

        print("random state = ",random_ls[i])

        acs_vrp = ACS_VRP(random_state=100)
        results_acs_vrp = get_analysis(acs_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
        random_node_analysis["acs_vrp"].append(results_acs_vrp)

        aco_vrp = ACO_VRP(random_state=100)
        results_aco_vrp = get_analysis(aco_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
        random_node_analysis["aco_vrp"].append(results_aco_vrp)

        bso_vrp = BSO_VRP(random_state=100)
        results_bso_vrp = get_analysis(bso_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
        random_node_analysis["bso_vrp"].append(results_bso_vrp)

        ga_vrp = GA_VRP(random_state=100)
        results_ga_vrp = get_analysis(ga_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
        random_node_analysis["ga_vrp"].append(results_ga_vrp)

        ts_vrp = TS_VRP(random_state=100)
        results_ts_vrp = get_analysis(ts_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
        random_node_analysis["ts_vrp"].append(results_ts_vrp)

        sa_vrp = SA_VRP(random_state=100)
        results_sa_vrp = get_analysis(sa_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
        random_node_analysis["sa_vrp"].append(results_sa_vrp)

        dka_vrp = DKA_VRP(random_state=100)
        results_dka_vrp = get_analysis(dka_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
        random_node_analysis["dka_vrp"].append(results_dka_vrp)
    
    doi_combi_analysis.append(random_node_analysis)

doi combination =  (0.15, 0.45, 0.77)
random state =  5
random state =  10
random state =  15
random state =  20
random state =  25
random state =  30
random state =  35
random state =  40
random state =  45
random state =  50
random state =  55
random state =  60
random state =  65
random state =  70
random state =  75
random state =  80
random state =  85
random state =  90
random state =  95
random state =  100
random state =  105
random state =  110
random state =  115
random state =  120
random state =  125
random state =  130
random state =  135
random state =  140
random state =  145
random state =  150
random state =  155
random state =  160
random state =  165
random state =  170
random state =  175
random state =  180
random state =  185
random state =  190
random state =  195
random state =  200
random state =  205
random state =  210
random state =  215
random state =  220
random state =  225
random state =  230
random state =  235
random state =  240
random state =  245
ra

random state =  10
random state =  15
random state =  20
random state =  25
random state =  30
random state =  35
random state =  40
random state =  45
random state =  50
random state =  55
random state =  60
random state =  65
random state =  70
random state =  75
random state =  80
random state =  85
random state =  90
random state =  95
random state =  100
random state =  105
random state =  110
random state =  115
random state =  120
random state =  125
random state =  130
random state =  135
random state =  140
random state =  145
random state =  150
random state =  155
random state =  160
random state =  165
random state =  170
random state =  175
random state =  180
random state =  185
random state =  190
random state =  195
random state =  200
random state =  205
random state =  210
random state =  215
random state =  220
random state =  225
random state =  230
random state =  235
random state =  240
random state =  245
random state =  250
doi combination =  (0.16, 0.65, 0.12)


In [44]:
doi_combi_analysis

[{'acs_vrp': [{'solution': [[74, 88, 2, 7, 50, 9, 15, 35, 93],
     [52, 22, 34, 4, 64, 10],
     [90, 55, 99, 72, 14, 48, 66, 83]],
    'fitness': 0.8263385668276974,
    'daily_fitness': [0.5944444444444447,
     0.498148148148148,
     0.4458333333333331],
    'days_needed': 3,
    'poi_included': 23,
    'avg_rating': 4.595652173913043,
    'total_tarif': 179500,
    'total_duration': 128040,
    'run_time': 2.1741812229156494},
   {'solution': [[46, 89, 12, 44, 18, 5, 57, 74],
     [39, 64, 48, 22, 2, 32, 36],
     [6, 67, 99, 34, 66, 10, 21, 49]],
    'fitness': 0.7749530327428877,
    'daily_fitness': [0.4770833333333333,
     0.4023809523809523,
     0.3583333333333334],
    'days_needed': 3,
    'poi_included': 23,
    'avg_rating': 4.569565217391304,
    'total_tarif': 142000,
    'total_duration': 129420,
    'run_time': 5.132083177566528},
   {'solution': [[46, 50, 74, 53, 2, 3, 22, 33],
     [64, 39, 81, 93, 43, 15, 36],
     [8, 20, 68, 48, 32, 5, 57]],
    'fitness': 0.7

In [45]:
result_to_xlsx_rd_doi(doi_combi_analysis,doi_combi,random_ls,"dka_random_doi_combi.xlsx")

## 87 POIs

In [11]:
def result_to_xlsx_all_poi(result,n,random_ls,filename):
    df_result = pd.DataFrame()
    for i in result:
        for j in range(len(result[i])):
            temp = pd.DataFrame(result[i][j],columns=["fitness","run_time"])
            temp['n'] = n[j]
            temp['method'] = i
            temp['random_seed'] = random_ls
            df_result = pd.concat([df_result,temp])
    df_result.to_excel(filename,index=False)

### All DOIs 1

In [6]:
n = [5,10,15,20,25,30,35,40,45,50,55,60,65,70,75,80,87]
run_time_analysis = {
    "acs_vrp": [],
    "aco_vrp": [],
    "bso_vrp": [],
    "ga_vrp": [],
    "ts_vrp": [],
    "sa_vrp": [],
    "dka_vrp": [],
}
max_runtime = 100*60 #100 minutes
for i in n:
    print("n = ",i)
    random_ls = list(range(5,(5*30)+1,5))
    random_node = {
        "acs_vrp": [],
        "aco_vrp": [],
        "bso_vrp": [],
        "ga_vrp": [],
        "ts_vrp": [],
        "sa_vrp": [],
        "dka_vrp": [],
    }
    
    for j in range(len(random_ls)):
        # setting
        hotel,tur,timematrix = generate_data(n=i,random_state=random_ls[j])
        travel_days = 3
        dwaktu,dtarif,drating = 1,1,1
        
        print("random_state : ",random_ls[j])
        
        acs_vrp = ACS_VRP(random_state=100)
        results_acs_vrp = get_analysis(acs_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
        random_node["acs_vrp"].append([results_acs_vrp[key] for key in ["fitness","run_time"]])

        aco_vrp = ACO_VRP(random_state=100)
        results_aco_vrp = get_analysis(aco_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
        random_node["aco_vrp"].append([results_aco_vrp[key] for key in ["fitness","run_time"]])

        bso_vrp = BSO_VRP(random_state=100)
        results_bso_vrp = get_analysis(bso_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
        random_node["bso_vrp"].append([results_bso_vrp[key] for key in ["fitness","run_time"]])

        ga_vrp = GA_VRP(random_state=100)
        results_ga_vrp = get_analysis(ga_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
        random_node["ga_vrp"].append([results_ga_vrp[key] for key in ["fitness","run_time"]])

        ts_vrp = TS_VRP(random_state=100)
        results_ts_vrp = get_analysis(ts_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
        random_node["ts_vrp"].append([results_ts_vrp[key] for key in ["fitness","run_time"]])

        sa_vrp = SA_VRP(random_state=100)
        results_sa_vrp = get_analysis(sa_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
        random_node["sa_vrp"].append([results_sa_vrp[key] for key in ["fitness","run_time"]])

        dka_vrp = DKA_VRP(random_state=100)
        results_dka_vrp = get_analysis(dka_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
        random_node["dka_vrp"].append([results_dka_vrp[key] for key in ["fitness","run_time"]])
    
    for method in run_time_analysis:
        run_time_analysis[method].append(random_node[method])

n =  5
random_state :  5
random_state :  10
random_state :  15
random_state :  20
random_state :  25
random_state :  30
random_state :  35
random_state :  40
random_state :  45
random_state :  50
random_state :  55
random_state :  60
random_state :  65
random_state :  70
random_state :  75
random_state :  80
random_state :  85
random_state :  90
random_state :  95
random_state :  100
random_state :  105
random_state :  110
random_state :  115
random_state :  120
random_state :  125
random_state :  130
random_state :  135
random_state :  140
random_state :  145
random_state :  150
n =  10
random_state :  5
random_state :  10
random_state :  15
random_state :  20
random_state :  25
random_state :  30
random_state :  35
random_state :  40
random_state :  45
random_state :  50
random_state :  55
random_state :  60
random_state :  65
random_state :  70
random_state :  75
random_state :  80
random_state :  85
random_state :  90
random_state :  95
random_state :  100
random_state :  105
rando

random_state :  150
n =  75
random_state :  5
random_state :  10
random_state :  15
random_state :  20
random_state :  25
random_state :  30
random_state :  35
random_state :  40
random_state :  45
random_state :  50
random_state :  55
random_state :  60
random_state :  65
random_state :  70
random_state :  75
random_state :  80
random_state :  85
random_state :  90
random_state :  95
random_state :  100
random_state :  105
random_state :  110
random_state :  115
random_state :  120
random_state :  125
random_state :  130
random_state :  135
random_state :  140
random_state :  145
random_state :  150
n =  80
random_state :  5
random_state :  10
random_state :  15
random_state :  20
random_state :  25
random_state :  30
random_state :  35
random_state :  40
random_state :  45
random_state :  50
random_state :  55
random_state :  60
random_state :  65
random_state :  70
random_state :  75
random_state :  80
random_state :  85
random_state :  90
random_state :  95
random_state :  100
rand

In [7]:
run_time_analysis

{'acs_vrp': [[[0.7043981481481482, 0.17060542106628418],
   [0.7103086419753084, 0.17373061180114746],
   [0.7047839506172839, 0.1346147060394287],
   [0.7191358024691361, 0.11027312278747559],
   [0.7155864197530865, 0.11882972717285156],
   [0.8951080246913582, 0.13130402565002441],
   [0.6951388888888891, 0.1571207046508789],
   [0.675771604938272, 0.13203096389770508],
   [0.7184413580246914, 0.1461470127105713],
   [0.7366512345679014, 0.12027359008789062],
   [0.7224537037037037, 0.1265573501586914],
   [0.7318672839506174, 0.11803507804870605],
   [0.706327160493827, 0.13926124572753906],
   [0.6888888888888887, 0.12955188751220703],
   [0.731327160493827, 0.1507551670074463],
   [0.7006481481481485, 0.1469740867614746],
   [0.6806327160493826, 0.12880373001098633],
   [0.6866512345679013, 0.13558220863342285],
   [0.7046296296296295, 0.14490175247192383],
   [0.6915123456790123, 0.17252254486083984],
   [0.7013734567901236, 0.12031173706054688],
   [0.6847222222222221, 0.120939

In [13]:
result_to_xlsx_all_poi(run_time_analysis,n,random_ls,"dka_various_numbers_pois_all_1.xlsx")

In [16]:
#continue
n = [5,10,15,20,25,30,35,40,45,50,55,60,65,70,75,80,87]
run_time_analysis = {
    "acs_vrp": [],
    "aco_vrp": [],
    "bso_vrp": [],
    "ga_vrp": [],
    "ts_vrp": [],
    "sa_vrp": [],
    "dka_vrp": [],
}
max_runtime = 100*60 #100 minutes
for i in n:
    print("n = ",i)
    random_ls = list(range(155,(5*50)+1,5))
    random_node = {
        "acs_vrp": [],
        "aco_vrp": [],
        "bso_vrp": [],
        "ga_vrp": [],
        "ts_vrp": [],
        "sa_vrp": [],
        "dka_vrp": [],
    }
    
    for j in range(len(random_ls)):
        # setting
        hotel,tur,timematrix = generate_data(n=i,random_state=random_ls[j])
        travel_days = 3
        dwaktu,dtarif,drating = 1,1,1
        
        print("random_state : ",random_ls[j])
        
        acs_vrp = ACS_VRP(random_state=100)
        results_acs_vrp = get_analysis(acs_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
        random_node["acs_vrp"].append([results_acs_vrp[key] for key in ["fitness","run_time"]])

        aco_vrp = ACO_VRP(random_state=100)
        results_aco_vrp = get_analysis(aco_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
        random_node["aco_vrp"].append([results_aco_vrp[key] for key in ["fitness","run_time"]])

        bso_vrp = BSO_VRP(random_state=100)
        results_bso_vrp = get_analysis(bso_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
        random_node["bso_vrp"].append([results_bso_vrp[key] for key in ["fitness","run_time"]])

        ga_vrp = GA_VRP(random_state=100)
        results_ga_vrp = get_analysis(ga_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
        random_node["ga_vrp"].append([results_ga_vrp[key] for key in ["fitness","run_time"]])

        ts_vrp = TS_VRP(random_state=100)
        results_ts_vrp = get_analysis(ts_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
        random_node["ts_vrp"].append([results_ts_vrp[key] for key in ["fitness","run_time"]])

        sa_vrp = SA_VRP(random_state=100)
        results_sa_vrp = get_analysis(sa_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
        random_node["sa_vrp"].append([results_sa_vrp[key] for key in ["fitness","run_time"]])

        dka_vrp = DKA_VRP(random_state=100)
        results_dka_vrp = get_analysis(dka_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
        random_node["dka_vrp"].append([results_dka_vrp[key] for key in ["fitness","run_time"]])
    
    for method in run_time_analysis:
        run_time_analysis[method].append(random_node[method])

n =  5
random_state :  155
random_state :  160
random_state :  165
random_state :  170
random_state :  175
random_state :  180
random_state :  185
random_state :  190
random_state :  195
random_state :  200
random_state :  205
random_state :  210
random_state :  215
random_state :  220
random_state :  225
random_state :  230
random_state :  235
random_state :  240
random_state :  245
random_state :  250
n =  10
random_state :  155
random_state :  160
random_state :  165
random_state :  170
random_state :  175
random_state :  180
random_state :  185
random_state :  190
random_state :  195
random_state :  200
random_state :  205
random_state :  210
random_state :  215
random_state :  220
random_state :  225
random_state :  230
random_state :  235
random_state :  240
random_state :  245
random_state :  250
n =  15
random_state :  155
random_state :  160
random_state :  165
random_state :  170
random_state :  175
random_state :  180
random_state :  185
random_state :  190
random_state :  1

In [17]:
run_time_analysis

{'acs_vrp': [[[0.7221141975308644, 0.20485758781433105],
   [0.7264126305792972, 0.1662287712097168],
   [0.6655864197530871, 0.1965022087097168],
   [0.7003086419753087, 0.16663432121276855],
   [0.7229938271604941, 0.21277832984924316],
   [0.7336419753086418, 0.15607309341430664],
   [0.7039351851851855, 0.1601564884185791],
   [0.7273919753086419, 0.13993430137634277],
   [0.6786265432098766, 0.2262251377105713],
   [0.7202932098765431, 0.20003223419189453],
   [0.7216820987654322, 0.17191433906555176],
   [0.7464576318742985, 0.13518595695495605],
   [0.7043209876543206, 0.2515721321105957],
   [0.7262808641975309, 0.22178006172180176],
   [0.6860339506172838, 0.15533208847045898],
   [0.7126543209876545, 0.16411328315734863],
   [0.6540895061728403, 0.2516474723815918],
   [0.713503086419753, 0.17514586448669434],
   [0.7217592592592593, 0.14107799530029297],
   [0.9045524691358026, 0.15718984603881836]],
  [[0.6704783950617285, 0.7410812377929688],
   [0.6924382716049383, 1.0776

In [19]:
result_to_xlsx_all_poi(run_time_analysis,n,random_ls,"dka_various_numbers_pois_all_1_v2.xlsx")

# Uji Statistik

In [20]:
def uji_statistik(first_data,second_data,alternative="greater",first_data_name="first",second_data_name="second"):
    # first_data is the target data, e.g. the results of Hybrid ACS-BSO 
    # second_data is a list of results from other method
    # the test is using Wilcoxon Rank-Sum Test
    second_data_name = ["second" for i in range(len(second_data))] if second_data_name == "second" else second_data_name
    for i in range(len(second_data)):
        statistic, p_value = stats.ranksums(first_data, second_data[i],alternative=alternative)
        print(f"Wilcoxon Rank-Sum Test ({first_data_name} vs. {second_data_name[i]}):")
        print("Test Statistic:", statistic)
        print("p-value:", p_value)
        print("")

## General performance

### All DOI 1

In [21]:
results_df = pd.read_excel("dka_random_nodes_alldoi_1.xlsx")
results_df.head()

,random_seed,fitness,fitness_1,fitness_2,fitness_3,poi_included,rating,tarif,total_duration,run_time,method
0,5,0.681458,0.709375,0.625561,0.650876,21,4.614286,37000,125100,4.769661,acs_vrp
1,10,0.644172,0.631843,0.602275,0.602765,20,4.580000,24500,124620,3.170985,acs_vrp
2,15,0.611977,0.603838,0.579350,0.546002,21,4.633333,81500,126480,2.801009,acs_vrp
3,20,0.678256,0.665319,0.629272,0.618132,22,4.590909,41000,122520,3.208303,acs_vrp
4,25,0.628851,0.631071,0.607775,0.587981,20,4.560000,86000,121860,5.879171,acs_vrp


In [28]:
grouped = results_df.groupby('method').agg({'fitness':'mean',
                                            'run_time':'mean'}).reset_index()
grouped.sort_values('fitness',ascending=False)

,method,fitness,run_time
1,acs_vrp,0.659326,3.746193
5,sa_vrp,0.658175,0.971710
3,dka_vrp,0.640759,0.462857
6,ts_vrp,0.640236,0.665049
0,aco_vrp,0.639912,2.691196
4,ga_vrp,0.630925,0.742191
2,bso_vrp,0.552943,0.378404


In [26]:
metrics = ["fitness","run_time"]

for metric in metrics:
    print("metric: ",metric)
    metrics = metric
    alternative = "greater" if metric in ["fitness","poi_included","rating"] else "less"
    first_data = results_df[results_df['method']=="dka_vrp"][metrics].values
    first_data_name = "dka_vrp"

    second_data = []
    second_data_name = []
    for method in results_df[~results_df['method'].isin(["dka_vrp"])]['method'].unique():
        second_data.append(results_df[results_df['method']==method][metrics].values)
        second_data_name.append(method)

    uji_statistik(first_data,second_data,alternative=alternative,first_data_name=first_data_name,second_data_name=second_data_name)

metric:  fitness
Wilcoxon Rank-Sum Test (dka_vrp vs. acs_vrp):
Test Statistic: -3.660618353867727
p-value: 0.999874196351087

Wilcoxon Rank-Sum Test (dka_vrp vs. aco_vrp):
Test Statistic: -0.40673537265196963
p-value: 0.6578988211912251

Wilcoxon Rank-Sum Test (dka_vrp vs. bso_vrp):
Test Statistic: 8.293265310174906
p-value: 5.5090605419404976e-17

Wilcoxon Rank-Sum Test (dka_vrp vs. ga_vrp):
Test Statistic: 1.8751190061243346
p-value: 0.030388176690415512

Wilcoxon Rank-Sum Test (dka_vrp vs. ts_vrp):
Test Statistic: -0.42052301240288387
p-value: 0.6629482890705894

Wilcoxon Rank-Sum Test (dka_vrp vs. sa_vrp):
Test Statistic: -3.433122297977642
p-value: 0.9997016635530885

metric:  run_time
Wilcoxon Rank-Sum Test (dka_vrp vs. acs_vrp):
Test Statistic: -8.617274844321392
p-value: 3.428320723737822e-18

Wilcoxon Rank-Sum Test (dka_vrp vs. aco_vrp):
Test Statistic: -8.617274844321392
p-value: 3.428320723737822e-18

Wilcoxon Rank-Sum Test (dka_vrp vs. bso_vrp):
Test Statistic: 1.8061808073

### Travel Duration 1

In [29]:
results_df = pd.read_excel("dka_random_nodes_dwaktu_1.xlsx")
results_df.head()

,random_seed,fitness,fitness_1,fitness_2,fitness_3,poi_included,rating,tarif,total_duration,run_time,method
0,5,0.648032,0.510880,0.424074,0.477778,23,4.556522,104500,121980,3.011678,acs_vrp
1,10,0.642361,0.487963,0.472685,0.440741,23,4.534783,196000,124920,4.785303,acs_vrp
2,15,0.631211,0.481481,0.438426,0.463657,23,4.604348,260500,129540,3.239691,acs_vrp
3,20,0.663850,0.483102,0.487037,0.457639,24,4.558333,283000,122400,3.555043,acs_vrp
4,25,0.626852,0.468287,0.459491,0.459259,22,4.500000,205500,124320,7.371741,acs_vrp


In [30]:
grouped = results_df.groupby('method').agg({'fitness':'mean',
                                            'run_time':'mean'}).reset_index()
grouped.sort_values('fitness',ascending=False)

,method,fitness,run_time
1,acs_vrp,0.653629,3.864807
0,aco_vrp,0.652414,2.674487
5,sa_vrp,0.651903,0.946983
6,ts_vrp,0.641319,0.648231
3,dka_vrp,0.637856,0.436884
4,ga_vrp,0.629941,0.713939
2,bso_vrp,0.585955,0.342127


In [31]:
metrics = ["fitness","run_time"]

for metric in metrics:
    print("metric: ",metric)
    metrics = metric
    alternative = "greater" if metric in ["fitness","poi_included","rating"] else "less"
    first_data = results_df[results_df['method']=="dka_vrp"][metrics].values
    first_data_name = "dka_vrp"

    second_data = []
    second_data_name = []
    for method in results_df[~results_df['method'].isin(["dka_vrp"])]['method'].unique():
        second_data.append(results_df[results_df['method']==method][metrics].values)
        second_data_name.append(method)

    uji_statistik(first_data,second_data,alternative=alternative,first_data_name=first_data_name,second_data_name=second_data_name)

metric:  fitness
Wilcoxon Rank-Sum Test (dka_vrp vs. acs_vrp):
Test Statistic: -3.4917197669190276
p-value: 0.9997600392624314

Wilcoxon Rank-Sum Test (dka_vrp vs. aco_vrp):
Test Statistic: -3.102218943955701
p-value: 0.9990396207947709

Wilcoxon Rank-Sum Test (dka_vrp vs. bso_vrp):
Test Statistic: 6.587044890999271
p-value: 2.243333307485938e-11

Wilcoxon Rank-Sum Test (dka_vrp vs. ga_vrp):
Test Statistic: 1.9233757452525344
p-value: 0.027216439076093498

Wilcoxon Rank-Sum Test (dka_vrp vs. ts_vrp):
Test Statistic: -0.8582805744944105
p-value: 0.804631223382131

Wilcoxon Rank-Sum Test (dka_vrp vs. sa_vrp):
Test Statistic: -3.029833835263401
p-value: 0.9987765584054343

metric:  run_time
Wilcoxon Rank-Sum Test (dka_vrp vs. acs_vrp):
Test Statistic: -8.617274844321392
p-value: 3.428320723737822e-18

Wilcoxon Rank-Sum Test (dka_vrp vs. aco_vrp):
Test Statistic: -8.617274844321392
p-value: 3.428320723737822e-18

Wilcoxon Rank-Sum Test (dka_vrp vs. bso_vrp):
Test Statistic: 2.2887481986517

### Tarif 1

In [32]:
results_df = pd.read_excel("dka_random_nodes_dtarif_1.xlsx")
results_df.head()

,random_seed,fitness,fitness_1,fitness_2,fitness_3,poi_included,rating,tarif,total_duration,run_time,method
0,5,0.809434,0.593082,0.592138,0.583648,21,4.604762,43000,128340,1.987593,acs_vrp
1,10,0.807227,0.575221,0.607817,0.598083,20,4.565000,29500,127380,4.700382,acs_vrp
2,15,0.791200,0.622184,0.550649,0.561034,21,4.623810,83500,129600,3.271467,acs_vrp
3,20,0.841992,0.639924,0.627036,0.550588,22,4.586364,39000,127800,5.548777,acs_vrp
4,25,0.780875,0.588576,0.555925,0.569949,21,4.528571,106000,126180,3.074423,acs_vrp


In [33]:
grouped = results_df.groupby('method').agg({'fitness':'mean',
                                            'run_time':'mean'}).reset_index()
grouped.sort_values('fitness',ascending=False)

,method,fitness,run_time
5,sa_vrp,0.825749,0.957705
1,acs_vrp,0.825103,3.643509
6,ts_vrp,0.807096,0.663548
3,dka_vrp,0.799982,0.406919
4,ga_vrp,0.794564,0.767411
0,aco_vrp,0.786884,2.803389
2,bso_vrp,0.661432,0.193773


In [34]:
metrics = ["fitness","run_time"]

for metric in metrics:
    print("metric: ",metric)
    metrics = metric
    alternative = "greater" if metric in ["fitness","poi_included","rating"] else "less"
    first_data = results_df[results_df['method']=="dka_vrp"][metrics].values
    first_data_name = "dka_vrp"

    second_data = []
    second_data_name = []
    for method in results_df[~results_df['method'].isin(["dka_vrp"])]['method'].unique():
        second_data.append(results_df[results_df['method']==method][metrics].values)
        second_data_name.append(method)

    uji_statistik(first_data,second_data,alternative=alternative,first_data_name=first_data_name,second_data_name=second_data_name)

metric:  fitness
Wilcoxon Rank-Sum Test (dka_vrp vs. acs_vrp):
Test Statistic: -4.5154520184244085
p-value: 0.9999968409074199

Wilcoxon Rank-Sum Test (dka_vrp vs. aco_vrp):
Test Statistic: 1.9923139440071056
p-value: 0.023168311354052607

Wilcoxon Rank-Sum Test (dka_vrp vs. bso_vrp):
Test Statistic: 8.59659338469502
p-value: 4.105865323277908e-18

Wilcoxon Rank-Sum Test (dka_vrp vs. ga_vrp):
Test Statistic: 0.8272583850548535
p-value: 0.20404531275840243

Wilcoxon Rank-Sum Test (dka_vrp vs. ts_vrp):
Test Statistic: -1.2684628570841088
p-value: 0.897683646580627

Wilcoxon Rank-Sum Test (dka_vrp vs. sa_vrp):
Test Statistic: -4.639540776182637
p-value: 0.9999982540788216

metric:  run_time
Wilcoxon Rank-Sum Test (dka_vrp vs. acs_vrp):
Test Statistic: -8.617274844321392
p-value: 3.428320723737822e-18

Wilcoxon Rank-Sum Test (dka_vrp vs. aco_vrp):
Test Statistic: -8.617274844321392
p-value: 3.428320723737822e-18

Wilcoxon Rank-Sum Test (dka_vrp vs. bso_vrp):
Test Statistic: 7.0110148133398

### Rating 1

In [35]:
results_df = pd.read_excel("dka_random_nodes_drating_1.xlsx")
results_df.head()

,random_seed,fitness,fitness_1,fitness_2,fitness_3,poi_included,rating,tarif,total_duration,run_time,method
0,5,0.826339,0.594444,0.498148,0.445833,23,4.595652,179500,128040,2.198029,acs_vrp
1,10,0.774953,0.477083,0.402381,0.358333,23,4.569565,142000,129420,5.177960,acs_vrp
2,15,0.738227,0.454762,0.320748,0.310544,22,4.640909,151500,126180,3.272198,acs_vrp
3,20,0.795721,0.550000,0.427778,0.377778,23,4.591304,235000,127980,1.410371,acs_vrp
4,25,0.750396,0.483333,0.395455,0.365152,21,4.566667,139000,127080,2.331297,acs_vrp


In [36]:
grouped = results_df.groupby('method').agg({'fitness':'mean',
                                            'run_time':'mean'}).reset_index()
grouped.sort_values('fitness',ascending=False)

,method,fitness,run_time
1,acs_vrp,0.796156,3.169225
5,sa_vrp,0.793384,0.963424
6,ts_vrp,0.785960,0.673597
3,dka_vrp,0.780811,0.407809
0,aco_vrp,0.775737,2.412886
4,ga_vrp,0.775300,0.661126
2,bso_vrp,0.708324,0.226886


In [37]:
metrics = ["fitness","run_time"]

for metric in metrics:
    print("metric: ",metric)
    metrics = metric
    alternative = "greater" if metric in ["fitness","poi_included","rating"] else "less"
    first_data = results_df[results_df['method']=="dka_vrp"][metrics].values
    first_data_name = "dka_vrp"

    second_data = []
    second_data_name = []
    for method in results_df[~results_df['method'].isin(["dka_vrp"])]['method'].unique():
        second_data.append(results_df[results_df['method']==method][metrics].values)
        second_data_name.append(method)

    uji_statistik(first_data,second_data,alternative=alternative,first_data_name=first_data_name,second_data_name=second_data_name)

metric:  fitness
Wilcoxon Rank-Sum Test (dka_vrp vs. acs_vrp):
Test Statistic: -2.323217298029047
p-value: 0.9899162590148257

Wilcoxon Rank-Sum Test (dka_vrp vs. aco_vrp):
Test Statistic: 0.6824881676702542
p-value: 0.24746516105881167

Wilcoxon Rank-Sum Test (dka_vrp vs. bso_vrp):
Test Statistic: 6.845563136328913
p-value: 3.808780670631827e-12

Wilcoxon Rank-Sum Test (dka_vrp vs. ga_vrp):
Test Statistic: 0.7686609161134681
p-value: 0.22104731630897156

Wilcoxon Rank-Sum Test (dka_vrp vs. ts_vrp):
Test Statistic: -0.854833664556682
p-value: 0.8036783807854333

Wilcoxon Rank-Sum Test (dka_vrp vs. sa_vrp):
Test Statistic: -1.95784484462982
p-value: 0.9748758892480844

metric:  run_time
Wilcoxon Rank-Sum Test (dka_vrp vs. acs_vrp):
Test Statistic: -8.617274844321392
p-value: 3.428320723737822e-18

Wilcoxon Rank-Sum Test (dka_vrp vs. aco_vrp):
Test Statistic: -8.617274844321392
p-value: 3.428320723737822e-18

Wilcoxon Rank-Sum Test (dka_vrp vs. bso_vrp):
Test Statistic: 5.659826117750289

## DOI Combination

In [38]:
results_df = pd.read_excel("dka_random_doi_combi.xlsx")
results_df.head()

,dwaktu,dtarif,drating,method,random_seed,fitness,fitness_1,fitness_2,fitness_3,poi_included,rating,tarif,total_duration,run_time
0,0.15,0.45,0.77,acs_vrp,5,0.826339,0.594444,0.498148,0.445833,23,4.595652,179500,128040,2.174181
1,0.15,0.45,0.77,acs_vrp,10,0.774953,0.477083,0.402381,0.358333,23,4.569565,142000,129420,5.132083
2,0.15,0.45,0.77,acs_vrp,15,0.738227,0.454762,0.320748,0.310544,22,4.640909,151500,126180,3.295432
3,0.15,0.45,0.77,acs_vrp,20,0.795721,0.550000,0.427778,0.377778,23,4.591304,235000,127980,1.357947
4,0.15,0.45,0.77,acs_vrp,25,0.750396,0.483333,0.395455,0.365152,21,4.566667,139000,127080,2.286925


In [39]:
results_df.groupby(['dwaktu','dtarif','drating','method']).agg({'fitness':'mean',
                                                               'run_time':'mean'})

fitness  run_time
dwaktu dtarif drating method                     
0.15   0.45   0.77    aco_vrp  0.775737  2.403636
                      acs_vrp  0.796156  3.142520
                      bso_vrp  0.708324  0.225677
                      dka_vrp  0.780811  0.407573
                      ga_vrp   0.775300  0.657392
...                                 ...       ...
0.96   0.21   0.96    bso_vrp  0.708324  0.241966
                      dka_vrp  0.780811  0.436503
                      ga_vrp   0.775300  0.702295
                      sa_vrp   0.793384  1.023080
                      ts_vrp   0.785960  0.706852

[70 rows x 2 columns]

In [40]:
random.seed(100)
doi_combi = [(np.round(random.uniform(0,1),2),np.round(random.uniform(0,1),2),np.round(random.uniform(0,1),2)) for i in range(10)]

In [42]:
for doi in doi_combi:
    print("doi combination : ",doi)
    metrics = ["fitness","run_time"]
    for metric in metrics:
        print("metric: ",metric)
        metrics = metric
        alternative = "greater" if metric in ["fitness","poi_included","rating"] else "less"
        first_data = results_df[(results_df['method']=="dka_vrp")&
                            (results_df['dwaktu']==doi[0])&
                            (results_df['dtarif']==doi[1])&
                            (results_df['drating']==doi[2])][metrics].values
        first_data_name = "dka_vrp"

        second_data = []
        second_data_name = []
        for method in results_df[~results_df['method'].isin(["dka_vrp"])]['method'].unique():
            second_data.append(results_df[(results_df['method']==method)&
                            (results_df['dwaktu']==doi[0])&
                            (results_df['dtarif']==doi[1])&
                            (results_df['drating']==doi[2])][metrics].values)
            second_data_name.append(method)

        uji_statistik(first_data,second_data,alternative=alternative,first_data_name=first_data_name,second_data_name=second_data_name)

doi combination :  (0.15, 0.45, 0.77)
metric:  fitness
Wilcoxon Rank-Sum Test (dka_vrp vs. acs_vrp):
Test Statistic: -2.323217298029047
p-value: 0.9899162590148257

Wilcoxon Rank-Sum Test (dka_vrp vs. aco_vrp):
Test Statistic: 0.6824881676702542
p-value: 0.24746516105881167

Wilcoxon Rank-Sum Test (dka_vrp vs. bso_vrp):
Test Statistic: 6.845563136328913
p-value: 3.808780670631827e-12

Wilcoxon Rank-Sum Test (dka_vrp vs. ga_vrp):
Test Statistic: 0.7686609161134681
p-value: 0.22104731630897156

Wilcoxon Rank-Sum Test (dka_vrp vs. ts_vrp):
Test Statistic: -0.854833664556682
p-value: 0.8036783807854333

Wilcoxon Rank-Sum Test (dka_vrp vs. sa_vrp):
Test Statistic: -1.95784484462982
p-value: 0.9748758892480844

metric:  run_time
Wilcoxon Rank-Sum Test (dka_vrp vs. acs_vrp):
Test Statistic: -8.617274844321392
p-value: 3.428320723737822e-18

Wilcoxon Rank-Sum Test (dka_vrp vs. aco_vrp):
Test Statistic: -8.617274844321392
p-value: 3.428320723737822e-18

Wilcoxon Rank-Sum Test (dka_vrp vs. bso_v

## All DOI

In [43]:
results_df1 = pd.read_excel("dka_various_numbers_pois_all_1.xlsx")
results_df2 = pd.read_excel("dka_various_numbers_pois_all_1_v2.xlsx")
results_df = pd.concat([results_df1,results_df2])
results_df = results_df.reset_index(drop=True)
results_df.head()

,fitness,run_time,n,method,random_seed
0,0.704398,0.170605,5,acs_vrp,5
1,0.710309,0.173731,5,acs_vrp,10
2,0.704784,0.134615,5,acs_vrp,15
3,0.719136,0.110273,5,acs_vrp,20
4,0.715586,0.118830,5,acs_vrp,25


In [45]:
results_df.groupby(['method','n']).mean()

fitness  run_time  random_seed
method  n                                  
aco_vrp 5   0.719153  0.255895        127.5
        10  0.673169  0.724553        127.5
        15  0.659874  1.297405        127.5
        20  0.665149  1.753226        127.5
        25  0.656030  2.105621        127.5
...              ...       ...          ...
ts_vrp  65  0.558926  1.208084        127.5
        70  0.556937  1.255552        127.5
        75  0.556266  1.264782        127.5
        80  0.550936  1.328494        127.5
        87  0.547804  1.432478        127.5

[119 rows x 3 columns]

In [46]:
n = results_df['n'].unique()
n.sort()
metric = "fitness"
alternative = "greater"

for i in n:
    print("n: ",i)
    first_data = results_df[(results_df['method']=="dka_vrp")&(results_df['n']==i)][metric].values
    first_data_name = "dka_vrp"

    second_data = []
    second_data_name = []
    for method in results_df[results_df['method'] != "dka_vrp"]['method'].unique():
        second_data.append(results_df[(results_df['method']==method)&(results_df['n']==i)][metric].values)
        second_data_name.append(method)

    uji_statistik(first_data,second_data,alternative=alternative,first_data_name=first_data_name,second_data_name=second_data_name)

n:  5
Wilcoxon Rank-Sum Test (dka_vrp vs. acs_vrp):
Test Statistic: 0.017234549688642783
p-value: 0.49312474980612636

Wilcoxon Rank-Sum Test (dka_vrp vs. aco_vrp):
Test Statistic: 0.2309429658278133
p-value: 0.4086795554597926

Wilcoxon Rank-Sum Test (dka_vrp vs. bso_vrp):
Test Statistic: 0.6514659782306972
p-value: 0.25737286603695325

Wilcoxon Rank-Sum Test (dka_vrp vs. ga_vrp):
Test Statistic: 0.04136291925274268
p-value: 0.48350328681658883

Wilcoxon Rank-Sum Test (dka_vrp vs. ts_vrp):
Test Statistic: 0.0
p-value: 0.5

Wilcoxon Rank-Sum Test (dka_vrp vs. sa_vrp):
Test Statistic: 0.15511094719778504
p-value: 0.4383669260116688

n:  10
Wilcoxon Rank-Sum Test (dka_vrp vs. acs_vrp):
Test Statistic: -0.41362919252742675
p-value: 0.6604271547348789

Wilcoxon Rank-Sum Test (dka_vrp vs. aco_vrp):
Test Statistic: 0.6273376086665973
p-value: 0.2652189779269818

Wilcoxon Rank-Sum Test (dka_vrp vs. bso_vrp):
Test Statistic: 1.4477021738459936
p-value: 0.07385018086324041

Wilcoxon Rank-Sum Te

In [47]:
n = results_df['n'].unique()
n.sort()
metric = "run_time"
alternative = "less"

for i in n:
    print("n: ",i)
    first_data = results_df[(results_df['method']=="dka_vrp")&(results_df['n']==i)][metric].values
    first_data_name = "dka_vrp"

    second_data = []
    second_data_name = []
    for method in results_df[results_df['method'] != "dka_vrp"]['method'].unique():
        second_data.append(results_df[(results_df['method']==method)&(results_df['n']==i)][metric].values)
        second_data_name.append(method)

    uji_statistik(first_data,second_data,alternative=alternative,first_data_name=first_data_name,second_data_name=second_data_name)

n:  5
Wilcoxon Rank-Sum Test (dka_vrp vs. acs_vrp):
Test Statistic: -8.617274844321392
p-value: 3.428320723737822e-18

Wilcoxon Rank-Sum Test (dka_vrp vs. aco_vrp):
Test Statistic: -8.617274844321392
p-value: 3.428320723737822e-18

Wilcoxon Rank-Sum Test (dka_vrp vs. bso_vrp):
Test Statistic: 5.983835651896774
p-value: 0.9999999989102839

Wilcoxon Rank-Sum Test (dka_vrp vs. ga_vrp):
Test Statistic: -8.617274844321392
p-value: 3.428320723737822e-18

Wilcoxon Rank-Sum Test (dka_vrp vs. ts_vrp):
Test Statistic: -8.617274844321392
p-value: 3.428320723737822e-18

Wilcoxon Rank-Sum Test (dka_vrp vs. sa_vrp):
Test Statistic: -8.617274844321392
p-value: 3.428320723737822e-18

n:  10
Wilcoxon Rank-Sum Test (dka_vrp vs. acs_vrp):
Test Statistic: -8.617274844321392
p-value: 3.428320723737822e-18

Wilcoxon Rank-Sum Test (dka_vrp vs. aco_vrp):
Test Statistic: -8.617274844321392
p-value: 3.428320723737822e-18

Wilcoxon Rank-Sum Test (dka_vrp vs. bso_vrp):
Test Statistic: 4.646434596058094
p-value: 0